# **TEAM CHALLENGUE: TOOLBOX**
## **Función: tipifica_variables**

Esta función permite clasificar automáticamente las variables de un DataFrame como:
- **Binaria**
- **Categórica**
- **Numérica Discreta**
- **Numérica Continua**

### **1. Definición de la función**

La función recibe:

- df: el DataFrame a analizar.

- umbral_categoria: número máximo de valores únicos para considerar una columna categórica.

- umbral_continua: porcentaje mínimo de valores únicos para considerar una columna como numérica continua.

```python
def tipifica_variables(df, umbral_categoria=10, umbral_continua=0.2):
```
### **2. Inicialización de resultados**

Se crea una lista vacía para guardar los resultados, y se calcula el número total de filas del DataFrame (n), necesario para calcular el porcentaje de cardinalidad.

```python
resultado = []
n = len(df)
```

### **3. Iteración por columnas del DataFrame**

Para cada columna:

- Se calcula la cardinalidad (número de valores únicos).

- Se calcula el porcentaje que representan esos valores únicos respecto al total de filas.

```python
for col in df.columns:
    cardinalidad = df[col].nunique()
    porcentaje_cardinalidad = round(cardinalidad / n, 2)
```

### **4. Clasificación de la variable**

Se aplica la lógica de clasificación:

- Si hay 2 valores únicos → Binaria
- Si hay menos de umbral_categoria → Categórica
- Si el porcentaje es alto → Numérica Continua
- Si no, se considera → Numérica Discreta

```python
if cardinalidad == 2:
        tipo = "Binaria"
    elif cardinalidad < umbral_categoria:
        tipo = "Categórica"
    elif porcentaje_cardinalidad >= umbral_continua:
        tipo = "Numerica Continua"
    else:
        tipo = "Numerica Discreta"
```

### **5. Guardar el resultado**

Se añade un diccionario con el nombre de la variable y su tipo sugerido a la lista resultado.

```python
resultado.append({
        "nombre_variable": col,
        "tipo_sugerido": tipo
    })
```

### **6. Devolver el resultado como DataFrame**

Finalmente, se transforma la lista de resultados en un DataFrame para facilitar su visualización y análisis posterior.

```python
return pd.DataFrame(resultado)
```

## **Función: get_features_cat_regression**

Esta función permite identificar qué variables categóricas del DataFrame están estadísticamente relacionadas con una variable numérica continua o discreta con alta cardinalidad.

Se basa en tests de significación estadística:
- **`ttest_ind`** para comparar medias cuando la variable categórica es binaria.
- **`f_oneway` (ANOVA)** para comparar medias entre más de dos grupos.

### **1. Validaciones de entrada**

Antes de hacer los cálculos, la función comprueba:

- Que el DataFrame sea válido
```python
if not isinstance(df, pd.DataFrame):
    print("'df' debe ser un DataFrame.")
    return None
```

- Que target_col está presente en el DataFrame.
```python
if target_col not in df.columns:
    print(f"La columna '{target_col}' no está en el DataFrame.")
    return None
```

- Que target_col es numérica.
```python
if not np.issubdtype(df[target_col].dtype, np.number):
    print(f"La columna '{target_col}' no es numérica.")
    return None
```

- Que el `pvalue` esté en el rango correcto (entre 0 y 1)
```python
if not (0 < pvalue < 1):
    print("'pvalue' debe estar entre 0 y 1.")
    return None
```

### **2. Comprobación de cardinalidad del target**

Se asegura que la variable objetivo tenga:

- Al menos 10 valores únicos

- Un 5% o más de variación respecto al total de filas
```python
cardinalidad = df[target_col].nunique()
porcentaje = cardinalidad / len(df)

if cardinalidad < 10 or porcentaje < 0.05:
    print(f"La variable '{target_col}' no tiene suficiente cardinalidad para considerarse continua.")
    print(f"Cardinalidad única: {cardinalidad} ({round(porcentaje * 100, 2)}%)")
    return None
```
Esto confirma que estamos ante una variable continua o discreta con alta cardinalidad.

### **3. Selección de variables categóricas**

Extrae las columnas categóricas del DataFrame. Si no hay ninguna, devuelve una lista vacía.
```python
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
if not cat_cols:
    print("No hay variables categóricas en el DataFrame.")
    return []
```

### **4. Iteración por variables categóricas**

Para cada variable categórica:

- Se identifican los niveles únicos (niveles) excluyendo nulos.

- Se agrupan los valores de target_col según cada nivel.
```python
relacionadas = []

for col in cat_cols:
    niveles = df[col].dropna().unique()
    grupos = [df[df[col] == nivel][target_col].dropna() for nivel in niveles]
```

### **5. Verificación de tamaño de muestra**

Si alguno de los grupos tiene menos de 2 datos, se salta esa variable para evitar errores en el test estadístico.
```python
 if any(len(grupo) < 2 for grupo in grupos):
        continue
```

### **6. Aplicación del test estadístico**

Se escoge automáticamente el test adecuado:

- Si hay 2 niveles: se aplica ttest_ind (test T de Student).

- Si hay más de 2 niveles: se aplica f_oneway (ANOVA).

- Si por algún motivo hay menos de 2 niveles válidos, se ignora.
```python
 try:
        if len(niveles) == 2:
            stat, p = ttest_ind(*grupos)
        elif len(niveles) > 2:
            stat, p = f_oneway(*grupos)
        else:
            continue
```

### **7. Filtrado por significancia**

Si el valor p es inferior al umbral definido (pvalue), se considera que hay una relación estadísticamente significativa entre esa variable categórica y la variable target, y se añade a la lista de resultado.
```python
if p < pvalue:
            relacionadas.append(col)
```

### **8. Gestión de errores**

Si ocurre cualquier error durante el test, se muestra el mensaje y se continúa con la siguiente variable sin detener la función.
```python
except Exception as e:
        print(f"Error evaluando la columna '{col}': {e}")
        continue
```

### **9. Resultado final**

Devuelve una lista con las variables categóricas que están significativamente relacionadas con la variable numérica objetivo, según el valor de `p-value` elegido.

Cuanto más bajo sea el p-value, más estricta es la prueba.
```python
return relacionadas
```
